# TensorflowTutorialMNIST_CNN


### Change the network arcitecture to a convolutional NN

Convolutional neural networks are used in state-of-the art image classifiaction networks. They learn a set of 2-d filters that are applied to the input image to generate meaningful features. 

For a theoretical introduction to how CNNs work, see the [Notebook on CNNs](http://nbviewer.jupyter.org/urls/maucher.home.hdm-stuttgart.de/nb/ML/ConvolutionNeuralNetworks.ipynb) from the Machine Learning lecture.

1. reshape the input data to 2-dimensional 28x28 images
2. replace the input layer with a [conv2d](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d)
3. add a [dropout](https://www.tensorflow.org/api_docs/python/tf/nn/dropout) layer followed by another conv2d layer and a final dropout layer before feeding the resulting feature maps into to the fully connected layer

| Teammember |                    |
|------------|--------------------|
| 1.         | Christopher Caldwell |
| 2.         | Fabian Müller      |
| 3.         | An Dang         |

In [2]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np

In [3]:
def cnn_model_fn(features, labels, mode):
    tf.set_random_seed(1234)
    img_input = tf.reshape(features["X"], [-1,28,28,1])
    labels = tf.dtypes.cast(labels, tf.int64)
    # Input layer - here as a conv2d layer #1
    conv1 = tf.layers.conv2d(inputs=img_input, filters=32, kernel_size=[5,5], padding="same", activation=tf.nn.relu)
    
    # Dropout layer #1
    dropout1 = tf.layers.dropout(inputs=conv1, rate=0.3, training=mode == tf.estimator.ModeKeys.TRAIN, seed=(1234))
    
    # Conv2d layer #2
    conv2 = tf.layers.conv2d(inputs=dropout1, filters=32, kernel_size=[5,5], padding="same", activation=tf.nn.relu)
      
    # Dropout layer #2
    dropout2 = tf.layers.dropout(inputs=conv2, rate=0.1, training=mode == tf.estimator.ModeKeys.TRAIN, seed=(1234))

    flat_layer = tf.reshape(dropout2, [-1, 28 * 28 * 32])
    
    # Dense layer
    dense = tf.layers.dense(inputs=flat_layer, units=1024, activation=tf.nn.relu)
    
    # Logits layer
    logits = tf.layers.dense(inputs=dense, units=10)
       
    # Compile the predictions in a dict and return EstimatorSpec as object
    predictions = {
        'classes': tf.argmax(input=logits, axis=1), 
        'probabilities': tf.nn.softmax(logits, name='softmax_tensor')
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # Calculate loss
    print(labels)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    # Training sgd
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # Add evaluation - accuracy metrics
    eval_metrice = {
        'accuracy': tf.metrics.accuracy(labels=labels, predictions=predictions['classes'])
    }
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metrice)

**Comment:**

In this section we build the CNN model funtion with `tf.estimator`. It is an high level tensorflow API and makes it easy to configure, train, and evaluate a variety of machine learning models. 

src: https://towardsdatascience.com/first-contact-with-tensorflow-estimator-69a5e072998d

In [4]:
from tensorflow.keras.datasets.mnist import load_data
(train_X, train_y),(test_X, test_y) = load_data()

# Normalize
train_X = train_X/np.float(255)
test_X = test_X/np.float(255)

In [5]:
# Estimator
clf = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="minist_convent_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'minist_convent_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10e300198>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [6]:
# Set up logging for predictions - softmax_tensor from above
tensor_to_log = {'probabilities': 'softmax_tensor'}

# Log after every 50 steps 
logging_hook = tf.train.LoggingTensorHook(tensors=tensor_to_log, every_n_iter=50)

In [8]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": train_X},
    y=train_y,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

# train one step and display the probabilties
clf.train(
    input_fn=train_input_fn,
    steps=10,
    hooks=[logging_hook])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Tensor("Cast:0", shape=(100,), dtype=int64)
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into minist_co

INFO:tensorflow:loss = 2.307138442993164, step = 1
INFO:tensorflow:Saving checkpoints for 10 into minist_convent_model/model.ckpt.
INFO:tensorflow:Loss for final step: 2.2925848960876465.


**Comment:**

In this section we train the model. Important is here the `hooks=[logging_hook]` in the `clf.train` function. It saves every 50 steps as checkpoint. If you abort your execution, you can jump right in the checkpoint. 

In [9]:
clf.train(input_fn=train_input_fn, steps=1000)

INFO:tensorflow:Calling model_fn.
Tensor("Cast:0", shape=(100,), dtype=int64)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from minist_convent_model/model.ckpt-10
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 10 into minist_convent_model/model.ckpt.
INFO:tensorflow:loss = 2.284815788269043, step = 11


KeyboardInterrupt: 

**Comment:**

just stop the process because my pc is burn. :D

In [9]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": test_X},
    y=test_y,
    num_epochs=1,
    shuffle=False)

eval_results = clf.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
Tensor("Cast:0", shape=(?,), dtype=int64)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-10T19:46:46Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from minist_convent_model\model.ckpt-20
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-10-19:47:26
INFO:tensorflow:Saving dict for global step 20: accuracy = 0.2706, global_step = 20, loss = 2.2507064
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20: minist_convent_model\model.ckpt-20
{'accuracy': 0.2706, 'loss': 2.2507064, 'global_step': 20}


**Comment:**

Here wie can evaluate the results. So we get 'accuracy': 0.2706, 'loss': 2.2507064, 'global_step': 20. 

In my opinion if we change our structure into a "normal" CNN model with a convo-layer, pooling layer etc. we can reach a better result.